In [1]:
import json
import time
import requests

In [2]:
# Get the PIN and AUTHORIZATION CODE -> this allows retireving the ACCESS TOKEN and REFRESH TOKEN
# After running this function you need to visit GISMO's ecobee portal and send insert PIN to authorize
# application (Transactive Control).
def EB_auth_code(api_key):
    params = {'response_type': 'ecobeePin', 'client_id': API_KEY, 'scope': 'smartRead'}
    auth_url = 'https://api.ecobee.com/authorize'
    # Request the PIN and convert the JSON results to a Python dictionary
    results = requests.get(auth_url, params=params).json()
    print(results)
    return results['code']


# Retrieving access token and refresh token for the first time (after running EB_auth_code)
def EB_access_token(api_key, auth_code):
#     header = {'Content-Type':'text/json;charset=UTF-8'}
    # parameters needed to request tokens
    params = {'grant_type': 'ecobeePin', 'client_id': api_key, 'code': auth_code}
    # Request the Tokens
    token_url = 'https://api.ecobee.com/token'
    response = requests.post(token_url, params=params)
    print(response)
    if response.status_code == requests.codes.ok:
        results = response.json()
        return [results['access_token'], results['refresh_token']]
    else:
        print('Error')
        return None

# Refresh the access token after it is expeired (can be used if access token is not expired)
def EB_refresh_token(access_token, refresh_token):
    REFRESH_TOKEN = refresh_token
    url_base = 'https://api.ecobee.com/token?grant_type=refresh_token&refresh_token='
    url_client = '&client_id='
    header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
    r = requests.post(url_base+REFRESH_TOKEN+url_client+API_KEY, headers=header)
    # r.json()
    # Saving Refresh token
    print(r.json())
    refresh_token = r.json()['refresh_token']
    access_token = r.json()['access_token']
    print('New Refresh_Token: ', refresh_token)
    print('New Access_Token: ', access_token)
    return[access_token, refresh_token]


def EB_hvac_mode(access_token, refresh_token, mode = "off"):
    # mode: heat, cool, auto, off
    header = {'Content-Type': 'application/json;charset=UTF-8', 'Authorization':'Bearer ' + access_token}
#     payload = {'json': '{"selection": {"selectionType":"registered","selectionMatch":""},"thermostat": {"settings":{"hvacMode":"off"}}}'}
    payload = {"selection":{"selectionType":"registered","selectionMatch":""},"thermostat":{"settings":{"hvacMode":mode}}}
    print(payload)
    url = "https://api.ecobee.com/1/thermostat?format=json"
    print(url)
    r = requests.post(url, json=payload, headers=header)
#     r = requests.post(url, body=payload, headers=header)
#     r = requests.request('POST', url, headers=header, params=None, json=payload)
    print(r)
    print(r.json())
    
    
def EB_hvac_temp(access_token, refresh_token, temp = 690):
    header = {'Content-Type': 'application/json;charset=UTF-8', 'Authorization':'Bearer ' + access_token}
    payload = {"selection": {"selectionType":"registered","selectionMatch":""},"functions": [{"type":"setHold","params":{"holdType":"nextTransition","heatHoldTemp":temp,"coolHoldTemp":temp}}]}
    url = "https://api.ecobee.com/1/thermostat?format=json"
    r = requests.post(url, json=payload, headers=header)
    print(r)
    print(r.json())
    
    
def EB_info(access_token, refresh_token):
    url = 'https://api.ecobee.com/1/thermostat'
    header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
    payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
    resp = requests.get(url, params=payload, headers=header)
    # print(resp.json())
    try:
        therm_time = resp.json()['thermostatList'][0]['thermostatTime']
        last_mod = resp.json()['thermostatList'][0]['lastModified']
        temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
        hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
        print('Thermostat_time: ', therm_time)
        print('Last_Modified: ', last_mod)
        print('Temperature: ', temp)
        print('Humidity: ', hum)
        print()
        print('------------')
        print()
    except KeyError:
        print('Key not present. Most likely token is expired. Refreshing and continuing...')
        # Call Refresh token
        tokens = EB_refresh_token(access_token, refresh_token)
        access_token = tokens[0]
        refresh_token = tokens[1]
        url = 'https://api.ecobee.com/1/thermostat'
        header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
        payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
        resp = requests.get(url, params=payload, headers=header)
        therm_time = resp.json()['thermostatList'][0]['thermostatTime']
        last_mod = resp.json()['thermostatList'][0]['lastModified']
        temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
        hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
        print('Thermostat_time: ', therm_time)
        print('Last_Modified: ', last_mod)
        print('Temperature: ', temp)
        print('Humidity: ', hum)
        print()
        print('------------')
        print()
    return [therm_time, temp, hum]

def control_heat(temp_setpoint, access_token, refresh_token):
    tokens = [access_token, refresh_token]
    data = EB_info(tokens[0], tokens[1])
    print('Current_temp: ', data[1])
    if data[1] < temp_setpoint:
        EB_hvac_mode(tokens[0], tokens[1],mode='heat')
        EB_hvac_temp(tokens[0], tokens[1], temp=temp_setpoint+60)
    else:
        EB_hvac_mode(tokens[0], tokens[1],mode='off')

In [3]:
# API key
api_key = 'Sp2ixgD56v9NB9hUozd17nYWJ9fAjrti'

# Tokens as of 02/09/20: 4:44pm PT 
# tokens[0] is the ACCESS TOKEN and expires in 1hr -> need to refresh
# tokens[1] is the REFRESH TOKEN. Is used to refresh the ACCESS TOKEN
tokens = ['ddopD8dKkhI3UlKsDF8pfAF5Cwtm3uoK', '1cjGD49Zp2jnEMcxkZfLWiqeMKBWEBTm']


In [4]:
# Dont need to run this
# auth_code = EB_auth_code(api_key)
# tokens = EB_access_token(api_key, auth_code)

# tokens = EB_refresh_token(tokens[0], tokens[1])
# print(tokens)
# data = EB_info(tokens[0], tokens[1])
# print(data)
EB_hvac_mode(tokens[0], tokens[1], mode="cool")
# EB_hvac_temp(tokens[0], tokens[1], temp=650)

{'selection': {'selectionType': 'registered', 'selectionMatch': ''}, 'thermostat': {'settings': {'hvacMode': 'cool'}}}
https://api.ecobee.com/1/thermostat?format=json
<Response [500]>
{'status': {'code': 4, 'message': 'Serialization error. Not permitted to update model: Thermostat'}}


In [29]:
# Control
for i in range(30):
    control_heat(700, tokens[0], tokens[1])
    time.sleep(10)

Thermostat_time:  2020-02-09 09:42:53
Last_Modified:  2020-02-09 17:18:08
Temperature:  668
Humidity:  34

------------

Current_temp:  668
<Response [500]>
{'status': {'code': 4, 'message': 'Serialization error. Not permitted to update model: Thermostat'}}
<Response [500]>
{'status': {'code': 11, 'message': 'Function error. Not permitted to execute function due to missing permissions. Function: setHold'}}


KeyboardInterrupt: 

In [177]:
# Reading temperature

url = 'https://api.ecobee.com/1/thermostat'
header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + tokens[0]}
payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
resp = requests.get(url, params=payload, headers=header)
# print(resp.json())
try:
    therm_time = resp.json()['thermostatList'][0]['thermostatTime']
    last_mod = resp.json()['thermostatList'][0]['lastModified']
    temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
    hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
    print('Thermostat_time: ', therm_time)
    print('Last_Modified: ', last_mod)
    print('Temperature: ', temp)
    print('Humidity: ', hum)
    print()
    print('------------')
    print()
except KeyError:
    print('Key not present. Most likely token is expired. Refreshing and continuing...')
    # Call Refresh token
    tokens = EB_refresh_token(access_token, refresh_token)
    access_token = tokens[0]
    refresh_token = tokens[1]
    url = 'https://api.ecobee.com/1/thermostat'
    header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
    payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
    resp = requests.get(url, params=payload, headers=header)
    therm_time = resp.json()['thermostatList'][0]['thermostatTime']
    last_mod = resp.json()['thermostatList'][0]['lastModified']
    temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
    hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
    print('Thermostat_time: ', therm_time)
    print('Last_Modified: ', last_mod)
    print('Temperature: ', temp)
    print('Humidity: ', hum)
    print()
#     print('------------')
#     print()


# print(response.url)
# print(response.text)


Thermostat_time:  2020-02-07 12:50:59
Last_Modified:  2020-02-07 20:49:08
Temperature:  677
Humidity:  48

------------



In [92]:
temp_list = []
thermostat_time_list = []
hum_list = []
last_modified_list = []
t = range(3)
for i in range(720):
    url = 'https://api.ecobee.com/1/thermostat'
    header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
    payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
    resp = requests.get(url, params=payload, headers=header)
    try:
        therm_time = resp.json()['thermostatList'][0]['thermostatTime']
        last_mod = resp.json()['thermostatList'][0]['lastModified']
        temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
        hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
        print('Thermostat_time: ', therm_time)
        print('Last_Modified: ', last_mod)
        print('Temperature: ', temp)
        print('Humidity: ', hum)
        print()
        print('------------')
        print()
    except KeyError:
        print('Key not present. Most likely token is expired. Refreshing and continuing...')
        # Call Refresh token
        tokens = EB_refresh_token(access_token, refresh_token)
        access_token = tokens[0]
        refresh_token = tokens[1]
        url = 'https://api.ecobee.com/1/thermostat'
        header = {'Content-Type':'text/json', 'Authorization':'Bearer ' + access_token}
        payload = {'json': '{"selection":{"selectionType":"registered","selectionMatch":"","includeRuntime":"true"}}'}
        resp = requests.get(url, params=payload, headers=header)
        therm_time = resp.json()['thermostatList'][0]['thermostatTime']
        last_mod = resp.json()['thermostatList'][0]['lastModified']
        temp = resp.json()['thermostatList'][0]['runtime']['actualTemperature']
        hum = resp.json()['thermostatList'][0]['runtime']['actualHumidity']
        print('Thermostat_time: ', therm_time)
        print('Last_Modified: ', last_mod)
        print('Temperature: ', temp)
        print('Humidity: ', hum)
        print()
        print('------------')
        print()
    temp_list.append(temp)
    thermostat_time_list.append(therm_time)
    hum_list.append(hum)
    last_modified_list.append(last_mod)
    time.sleep(10)

Thermostat_time:  2020-02-06 22:02:28
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:02:39
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:02:50
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:03:00
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:03:11
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:03:22
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:03:33
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:03:44
Last_Modified:  2020-02-06 18:05:08
Temperature:  619
Humidity:  52

------------

Thermostat_time:  2020-02-06 22:

Thermostat_time:  2020-02-06 22:14:33
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:14:43
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:14:54
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:15:04
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:15:15
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:15:25
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:15:36
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:15:47
Last_Modified:  2020-02-07 06:11:08
Temperature:  619
Humidity:  53

------------

Thermostat_time:  2020-02-06 22:

Thermostat_time:  2020-02-06 22:26:33
Last_Modified:  2020-02-07 06:21:38
Temperature:  634
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:26:44
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:26:55
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:27:05
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:27:16
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:27:26
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:27:37
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:27:48
Last_Modified:  2020-02-07 06:21:38
Temperature:  631
Humidity:  51

------------

Thermostat_time:  2020-02-06 22:

Thermostat_time:  2020-02-06 22:38:35
Last_Modified:  2020-02-07 06:21:38
Temperature:  654
Humidity:  50

------------

Thermostat_time:  2020-02-06 22:38:46
Last_Modified:  2020-02-07 06:21:38
Temperature:  654
Humidity:  50

------------

Thermostat_time:  2020-02-06 22:38:56
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:39:08
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:39:18
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:39:29
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:39:39
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:39:50
Last_Modified:  2020-02-07 06:21:38
Temperature:  662
Humidity:  49

------------

Thermostat_time:  2020-02-06 22:

Thermostat_time:  2020-02-06 22:50:35
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:50:46
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:50:56
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:51:07
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:51:17
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:51:28
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:51:39
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:51:49
Last_Modified:  2020-02-07 06:21:38
Temperature:  669
Humidity:  48

------------

Thermostat_time:  2020-02-06 22:

...

Thermostat_time:  2020-02-06 23:14:06
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:14:17
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:14:27
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:14:38
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:14:48
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:14:59
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:15:09
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:15:20
Last_Modified:  2020-02-07 07:00:08
Temperature:  671
Humidity:  46

------------

Thermostat_time:  2020-02-06 23:

Thermostat_time:  2020-02-06 23:26:06
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:26:16
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:26:27
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:26:37
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:26:48
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:26:58
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:27:09
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:27:20
Last_Modified:  2020-02-07 07:00:08
Temperature:  658
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:

Thermostat_time:  2020-02-06 23:38:05
Last_Modified:  2020-02-07 07:00:08
Temperature:  652
Humidity:  49

------------

Thermostat_time:  2020-02-06 23:38:16
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:38:26
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:38:37
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:38:47
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:38:58
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:39:08
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:39:19
Last_Modified:  2020-02-07 07:00:08
Temperature:  657
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:

Thermostat_time:  2020-02-06 23:50:03
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:50:14
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:50:24
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:50:35
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:50:45
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:50:56
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:51:06
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:51:17
Last_Modified:  2020-02-07 07:00:08
Temperature:  665
Humidity:  48

------------

Thermostat_time:  2020-02-06 23:

Thermostat_time:  2020-02-07 00:01:31
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:01:41
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:01:52
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:02:02
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:02:13
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:02:23
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:02:34
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:02:45
Last_Modified:  2020-02-07 07:00:08
Temperature:  655
Humidity:  48

------------

Thermostat_time:  2020-02-07 00:

In [96]:
import csv
with open('ecobee_Temp_data.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(temp_list)
    
with open('ecobee_Timestamp_data.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(thermostat_time_list)
    